In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import pearsonr
from sklearn.covariance import GraphicalLassoCV
from networkx.generators.community import gaussian_random_partition_graph
import scipy.stats as stats
from scipy.stats import ttest_ind
import nibabel as nib 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import glob as glob
import math


#set random generator outside loop 
#seed everytrhing once at begibning and dont touch it 
np.random.seed(3)

#### For tau graphs, just do these steps, but on Tau suvr, stratify by amyloid status

- For stratifying by amyloid status,  Royse 2021 Validation of PET positivity thresholds in centiloids established that 21 (or 20) is about so use this

#### General steps 
#### 1. Read mri_pet/pet_prediciton_mldataframe.csv (which has info on gmv, amyloid centilod, and demographics)
- these are ~ 800 ADNI patients 
-  df['CENTILOIDS'] has amyloid centiloid value for now can just 24 as the threshold for amyloid pos and neg patients
- it also has freesurfer volumes by region 

- dx 1= nc, 2 = mci, 3 = AD 

In [2]:
df = pd.read_csv("/home/mlauber/mci_mri_graph/construct_ggm/ml_dataframe.csv")

In [3]:
#look for and load in the A4 dataset because I think it has more patients

In [4]:
print(len(df))
print(df.columns)

793
Index(['RID', 'PTGENDER', 'PTEDUCAT', 'PTTLANG', 'PTETHCAT', 'PTRACCAT',
       'AGE_MRI', 'DIAGNOSIS', 'Q1SCORE', 'Q4SCORE',
       ...
       'MMSE3', 'MMSE4', 'MMSE5', 'MMSE6', 'MMSE7', 'META_TEMPORAL',
       'MESIAL_TEMPORAL', 'TEMPO_PARIETAL', 'FRONTAL', 'NEOCORTICAL'],
      dtype='object', length=202)


- out of 793 patients 
- if cut off is 24 then 
- 339 amy pos
- 454 amy neg

- if cut off is 40 then 
- 256 pos : if I do quartiles then this is 64 patients in each quartile, could be kinda small, maybe use this as the replication dataset if possible 
- 526 neg 

In [11]:
#separate into pos and neg 
amy_pos = df[df['CENTILOIDS']>= 18]
amy_neg = df[df['CENTILOIDS']< 18]

# #if I use a stricter cut off of 40
# amy_pos = df[df['CENTILOIDS']>= 40]
# amy_neg = df[df['CENTILOIDS']< 40]
print(len(amy_pos))
print(len(amy_neg))

371
422


In [12]:
371 / 4

92.75

#### 2. Run and train separate graphical models for amyloid+ and amyloid- populations

##### I. Prepare data steps:
- a. need drop all non brain region vol cols - keep only brain regions 
- also drop the left and right ones 

In [7]:
#do some reg ex search for _VOLUME
def filter_regions(df):
    filtered_df = df.filter(regex='_VOLUME$')
    return filtered_df

amy_pos_reg = filter_regions(amy_pos)
amy_neg_reg = filter_regions(amy_neg)
amy_pos_reg.to_csv('amy_pos_regions.csv')
amy_neg_reg.to_csv('amy_neg_reg.csv')
print(amy_neg_reg.columns)

Index(['META_TEMPORAL_VOLUME', 'CTX_ENTORHINAL_VOLUME',
       'INFERIOR_CEREBGM_VOLUME', 'HEMIWM_VOLUME', 'BRAIN_STEM_VOLUME',
       'CHOROID_VOLUME', 'MIDDLEFR_VOLUME', 'ORBITOFR_VOLUME', 'PARSFR_VOLUME',
       'CTX_BANKSSTS_VOLUME',
       ...
       'LEFT_CAUDATE_VOLUME', 'LEFT_PALLIDUM_VOLUME', 'LEFT_PUTAMEN_VOLUME',
       'LEFT_THALAMUS_PROPER_VOLUME', 'RIGHT_ACCUMBENS_AREA_VOLUME',
       'RIGHT_AMYGDALA_VOLUME', 'RIGHT_CAUDATE_VOLUME',
       'RIGHT_PALLIDUM_VOLUME', 'RIGHT_PUTAMEN_VOLUME',
       'RIGHT_THALAMUS_PROPER_VOLUME'],
      dtype='object', length=110)


In [8]:
print(len(amy_pos.columns))

202


In [9]:
#drop left and right because some are redudant 
#something like if LEFT_ RIGHT_ or LH or RH
def filter_left_right(df):
    columns_to_keep = [col for col in df.columns if 'LEFT' not in col and 'RIGHT' not in col and 'LH' not in col and 'RH' not in col]
    return df[columns_to_keep]

# Assuming you have data in amy_pos_reg and amy_neg_reg
amy_pos_reg_comb = filter_left_right(amy_pos_reg)
amy_neg_reg_comb = filter_left_right(amy_neg_reg)

amy_pos_reg_comb.to_csv('amy_pos_reg_dropleftright.csv')
amy_neg_reg_comb.to_csv('amy_neg_reg_dropleftright.csv')

print(amy_pos_reg_comb.columns)
print(len(amy_pos_reg_comb.columns))



Index(['META_TEMPORAL_VOLUME', 'INFERIOR_CEREBGM_VOLUME', 'HEMIWM_VOLUME',
       'BRAIN_STEM_VOLUME', 'CHOROID_VOLUME', 'MIDDLEFR_VOLUME',
       'ORBITOFR_VOLUME', 'PARSFR_VOLUME', 'CTX_BANKSSTS_VOLUME',
       'CTX_CAUDALANTERIORCINGULATE_VOLUME', 'CTX_CUNEUS_VOLUME',
       'CTX_FUSIFORM_VOLUME', 'CTX_INFERIORPARIETAL_VOLUME',
       'CTX_INFERIORTEMPORAL_VOLUME', 'CTX_INSULA_VOLUME',
       'CTX_ISTHMUSCINGULATE_VOLUME', 'CTX_LATERALOCCIPITAL_VOLUME',
       'CTX_LINGUAL_VOLUME', 'CTX_MIDDLETEMPORAL_VOLUME',
       'CTX_PARACENTRAL_VOLUME', 'CTX_PARAHIPPOCAMPAL_VOLUME',
       'CTX_PERICALCARINE_VOLUME', 'CTX_POSTCENTRAL_VOLUME',
       'CTX_POSTERIORCINGULATE_VOLUME', 'CTX_PRECENTRAL_VOLUME',
       'CTX_PRECUNEUS_VOLUME', 'CTX_ROSTRALANTERIORCINGULATE_VOLUME',
       'CTX_SUPERIORFRONTAL_VOLUME', 'CTX_SUPERIORPARIETAL_VOLUME',
       'CTX_SUPERIORTEMPORAL_VOLUME', 'CTX_SUPRAMARGINAL_VOLUME',
       'CTX_TEMPORALPOLE_VOLUME', 'CTX_TRANSVERSETEMPORAL_VOLUME',
       'ACCUMBENS_ARE

- b. find out if there are any regions which have 0 volume or rows where there is no entry 

- c. normalize the data 

- d. check that there are no null values 

##### II. Create bootstrap samples and run model for amy pos

- a. create boot strap samples 

- b. run model 

- c. calc metrics 

##### III. Create bootstrap samples and run model for amy neg

- a. create boot strap samples 

- b. run model 

- c. calc metrics 

##### IV. Compare graph metrics using t test 

##### Later can Use GMM (2 components) to stratify patients into amyloid pos and neg 
- example code in mri_pet/pet_predicition_mldataframe.csv
- do thresholding on normal cognition 